<h1>ASL Detector Training</h1>
<p>Created by Alex Pereira</p>

<h2>Preprocessing</h2>

In [ ]:
# Import Libraries
import os
import numpy as np
import tensorflow as tf
from   tensorflow.keras.utils  import to_categorical
from   sklearn.model_selection import train_test_split

In [ ]:
# Gets variables for data collection
DATA_PATH    = os.path.join("MP_Data")
action_list  = np.load("actions.npy")  # List of gestures
num_videos   = 30  # Number of videos
video_frames = 30  # Frames per video

In [ ]:
# Creates a label map
label_map = {label:num for num, label in enumerate(action_list)}

In [ ]:
sequences, labels = [], []

# 
for action in action_list:
    # 
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        # 
        window = []

        # 
        for num_frame in range(0, video_frames):
            frame = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(num_frame)))
            window.append(frame)

        #
        sequences.append(window)

        # 
        labels.append(label_map[action])

In [ ]:
# Creates a numpy array
X = np.array(sequences)

# Stores the labels in an array
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)

<h2>Model Creation</h2>

In [ ]:
# Import model building components
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
# Create log directory
log_dir = os.path.join('Logs')

# Sets the location for TensorFlow to save logs
tb_callback = TensorBoard(log_dir = log_dir)

In [ ]:
# Makes a sequential model
model = Sequential()

In [ ]:
# Adds layers to the model
model.add(LSTM(64,  return_sequences = True , activation = 'relu', input_shape = (30,1662)))
model.add(LSTM(128, return_sequences = True , activation = 'relu'))
model.add(LSTM(64,  return_sequences = False, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(action_list.shape[0], activation = 'softmax'))

In [ ]:
# Print a model summary
model.summary()

In [ ]:
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs = 5000, callbacks = [tb_callback])

In [ ]:
model.save("actions.h5")